In [ ]:
### Versuch, etwas mit Whisper zu bauen:

In [ ]:
#und hier noch mit Whisper, https://github.com/ggml-org/whisper.cpp

In [ ]:
# https://mistral.ai/news/voxtral

In [ ]:
pip install openai-whisper

In [ ]:
import whisper
import tempfile
import requests
import os

# Whisper-Modell laden (nur 1x zu Beginn!)
whisper_model = whisper.load_model("base")

def transcribe_video(video_url: str) -> str:
    try:
        # Temporäre Datei
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as tmp_file:
            response = requests.get(video_url)
            tmp_file.write(response.content)
            tmp_path = tmp_file.name

        # Transkription
        result = whisper_model.transcribe(tmp_path)
        os.remove(tmp_path)
        return result["text"]

    except Exception as e:
        return f"[Fehler bei Transkription: {str(e)}]"


In [ ]:
"transcript": transcribe_video(video_url) if has_video and video_url else alt_text.strip() if alt_text else "",

In [31]:
import pandas as pd
fp = pd.read_csv("factbase_posts_playwright9.csv")
#print(fpdp.tail(10))
#print(fpp[600-604])
print(fp.shape)

(82252, 4)


In [1]:
from pytube import YouTube
import moviepy.editor as mp
import torch
from pyannote.audio import Pipeline
from whispercpp import Whisper, api
import os
import numpy as np
from pydub import AudioSegment
import ffmpeg

ModuleNotFoundError: No module named 'moviepy.editor'

In [2]:
from moviepy.editor import VideoFileClip

clip = VideoFileClip("testvideo.mp4")
clip = clip.subclip(0, 5)  # Nur die ersten 5 Sekunden
clip.audio.write_audiofile("output.wav")


ModuleNotFoundError: No module named 'moviepy.editor'

In [4]:
pip install imageio-ffmpeg

Note: you may need to restart the kernel to use updated packages.


In [3]:
import moviepy
print(moviepy.__file__)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/moviepy/__init__.py


In [5]:
import sys
print(sys.executable)


/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10


In [3]:
!pip install moviepy

In [37]:
pip install moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 5.0 MB/s  0:00:05 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [moviepy]m3/4 [moviepy]ffmpeg]
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install pytube


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
np.NAN = np.nan
#Lade das YouTube-Video herunter und extrahiere das Audio
def download_youtube_audio(url, output_path="audio.wav"):
    yt = YouTube(url)
    video = yt.streams.filter(only_audio=True).first()
    out_file = video.download(output_path='.')
    base, ext = os.path.splitext(out_file)
    new_file = base + '.wav'
    audio_clip = mp.AudioFileClip(out_file)
    audio_clip.write_audiofile(new_file)
    os.remove(out_file) # Entferne die ursprüngliche Datei
    
    
    # Resample das Audio auf 16kHz
    audio = AudioSegment.from_wav(new_file)
    audio = audio.set_frame_rate(16000)
    audio.export(new_file, format="wav")

    return new_file

# Beispiel-YouTube-URL (bitte durch deine URL ersetzen)
youtube_url = "https://www.youtube.com/watch?v=msNGQfMUBGQ"
audio_file = download_youtube_audio(youtube_url)

# Initialisiere die pyannote Pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")

# Schicke die Piipeline zur GPU (falls verfügbar)
pipeline.to(torch.device("cuda"))

# Wende die vortrainierte Pipeline an
diarization = pipeline(audio_file)

# Initialisiere Whisper
w = Whisper.from_pretrained('./ggml-large-v3.bin')
print(w) # Überprüfe, ob das Modell korrekt geladen ist

In [ ]:
import torch
import torchaudio
from torchaudio.trnsforms import MelSpectrogram

# Lade die Audio-Datei und konvertiere sie in ein Numpy-Array
def load_audio(file, sample_rate=16000):
    waveform, sr = torchaudio.load(file)
    if sr != sample_rate:
        transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        waveform = transform(waveform)
    return waveform.squeeze().numpy()
    
# Erzeuge ein Mel-Spektrogramm
def generate_mel_spectrogram(audio, sample_rate=16000, n_mels=128):
    mel_spectrogrm = MelSpectrogram(sample_rate=sample_rate, n_fft=400, hop_length=160, n_mels=n_mels)
    audio_tensor = torch.from_numpy(audio)
    mel_spec = mel_spectrogram(audio_tensor)
    return mel_spec.numpy()

# Transkribiere einen Audio-Chunk
def transcribe_audio_chunk(audio_chunk, sample_rate=16000):
    audio_array = np.array(audio_chunk.get_array_of_samples()).astype(np.float32)/ 32768.0
    mel_spec = generate_mel_spectrogram(audio_array, sample_rate)
    if mel_spec.shape[0] != 128:
        raise ValueError(f"Expected 128 Mel bands, but got {mel_spec.shape[0]}")
        
    audio_list = mel_spec.flatten().tolist() # Convert to list
    result = w.transcribe(audio_list)
    return w.extract_text(result)

# Der Rest deines Codes bleibt gleich


# Transkribiere die Audio-Chunks basierend auf Diarisierungsergebnissen
def transcribe_diarized_chunks(audio_file, diarization):
    audio = AudioSegment.from_wav(audio_file)
    combined_results = []
    
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        start_time = turn.start * 1000 # in milliseconds
        end_time = turn.end * 1000 # in milliseconds
        speaker_label = f"speaker_{speaker}"
        audio_chunk = audio[start_time:end_time]
        transcript_segment = transcribe_audio_chunk(audio_chunk)
        
        combined_results.append((start_time / 1000, end_time / 1000, speaker_label, transcript_segment))
        return combined_results
# Transkribiere die Chunks
combined_results = transcribe_diarized_chunks(audio_file, diarization)

# Ausgabe der kombinierten Ergebnisse
for start, end, speaker, transcript in combined_results:
    print(f"start={start:.1f}s stop={end:.1f}s {speaker}:{transcript}")
    
    
